<img src="https://raw.githubusercontent.com/Ironhack-Data-Madrid-Marzo-2021/W7-Kaggle_competition/master/images/PORTADA.jpg">

# Import libraries

In [1]:
import numpy as np
import pandas as pd
import src.limpieza as lm
import pickle

# Import Data


<img width=400 src="https://cdn.dribbble.com/users/576057/screenshots/3299406/collecting_data_www.ccccccc.dk_motionstudio.gif">


In [2]:
test = pd.read_csv("data/test_limpio_s.csv")
train = pd.read_csv("data/train_limpio_s.csv")

In [3]:
test = pd.read_csv("data/test_limpio.csv")
train = pd.read_csv("data/train_limpio.csv")

# Create target and feature columns

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X = train.drop(["id","price"], axis=1)
y = train["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [6]:
y_train.shape


(32364,)

In [7]:
X_train.shape

(32364, 7)

In [8]:
X_test.shape


(8091, 7)

# Best models

In [9]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier

In [10]:
models = {"ridge":Ridge(),
         "lasso":Lasso(),
         "sgd":SGDRegressor(),
         "KNN":KNeighborsRegressor(),
         "gradient":GradientBoostingRegressor(),
          "Rfr" : RandomForestRegressor()
         }

In [11]:
for name, model in models.items():
    print(f"Trainning model -----> {name}")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("MSE: ", metrics.mean_squared_error(y_test,y_pred))
    
    print("Model finished")

Trainning model -----> ridge
MSE:  0.12383528304217044
Model finished
Trainning model -----> lasso
MSE:  1.0295346595227768
Model finished
Trainning model -----> sgd
MSE:  3.8085625161159355e+32
Model finished
Trainning model -----> KNN
MSE:  1.1785317285823755
Model finished
Trainning model -----> gradient
MSE:  0.01252300783702609
Model finished
Trainning model -----> Rfr
MSE:  0.01087976025920158
Model finished


Best models to predict the diamond price are **Gradient Boosting** and **Random Forest** 

# Parameter Tuning
Now that we have found the most efficient model, let's try the different `hyperparameters` to improve it.

<img widht=300 src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTKDKAY6dn5D25dF8ixQ5FMYGvv34c_6Gxzfw&usqp=CAU">

In [14]:
from sklearn.model_selection import GridSearchCV

In [15]:
best

NameError: name 'best' is not defined

In [16]:
param = {
    "max_depth" : [7,8,9],
    "max_features" : [6,7,8],
    "min_samples_split" : [100,110,120]
}

In [17]:
parameters = {
 'max_depth': [6,7],
 'max_features': ['auto'],
 'min_samples_leaf': [1, 2,3],
 'min_samples_split': [60,80],
 'n_estimators': [150,180]}

In [18]:
parameters = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [19]:
parameters = {
 'max_depth': [7,8,9,11],
 'max_features': ['auto'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [90,100,110,120],
 'n_estimators': [180,200,220]}

In [ ]:
gs = GridSearchCV(estimator = GradientBoostingRegressor(),
                  param_grid = parameters,
                 cv = 5,
                 verbose = 3,
                 scoring = "neg_mean_squared_error",
                 return_train_score=True)

In [ ]:
gs.fit(X_train, y_train)

In [24]:
gs_results = pd.DataFrame(gs.cv_results_)

In [23]:
gs_results = gs_results[["param_max_depth","param_max_features","param_n_estimators","param_min_samples_leaf","param_min_samples_split","mean_test_score","mean_train_score"]]

KeyError: "['param_max_depth', 'param_min_samples_split', 'param_min_samples_leaf', 'param_max_features'] not in index"

In [25]:
gs_results.sort_values("mean_test_score", ascending = False).head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,12.540415,1.195398,0.04671,0.003633,500,{'n_estimators': 500},-0.01019,-0.010388,-0.01052,-0.010398,...,-0.01029,0.000198,1,-0.009317,-0.009189,-0.009185,-0.009353,-0.009274,-0.009264,0.000067


In [26]:
best = gs.best_estimator_
best

GradientBoostingRegressor(n_estimators=500)

In [27]:
metrics.mean_squared_error(best.predict(X_test),y_test)

0.009927655261268802

In [ ]:
pickle.dump(best,open("param/gradient_b7_00957", "wb"))

# Best Parameters

<img width=300 src="https://i.pinimg.com/originals/01/46/c2/0146c226dbab4fa907566c6cfce03c5c.gif">

#### Submit2
GradientBoostingRegressor(max_depth=7, max_features=7, min_samples_split=100)
* MSE = 0.009384625525081075



#### Submit3
GradientBoostingRegressor(max_depth=9, max_features=7, min_samples_split=120)
* MSE = 0.009289453634141622



#### Submit3
GradientBoostingRegressor(max_depth=8, max_features=8, min_samples_split=120)
* MSE = 0.009245606227868533



#### Submit4
GradientBoostingRegressor(max_depth=8, max_features=7, min_samples_split=110)
* MSE = 0.009237581613435906



#### prueba 5
GradientBoostingRegressor(max_depth=8, max_features=7, min_samples_split=100)
* MSE = 0.009120234132274974



In [ ]:
from sklearn import *

# Test Prices

In [ ]:
# load the model from disk
loaded_model = pickle.load(open("param/rf_b3_00923", 'rb'))
loaded_model

In [ ]:
gbr = loaded_model

In [ ]:
gbr.fit(X_train, y_train)

In [ ]:
test = pd.read_csv("data/test_limpio.csv")
y_pred = gbr.predict(test)

In [ ]:
test["price"] = y_pred
test.head()

In [ ]:
submit = test[["id","price"]]
submit.set_index("id", inplace=True)

In [ ]:
submit.head()

In [ ]:
submit.to_csv("submitions/submit8.csv")

<img width=700 src="https://i.pinimg.com/originals/cc/3f/b6/cc3fb61df5200eca518fe22adf03e073.gif">